Analysis of a MIDI file, check for number of instruments and note structure

In [1]:
import pretty_midi
import glob

In [2]:
midi_data = pretty_midi.PrettyMIDI(r"..\midi\downloaded_midi\3_SATZ_AUS_DER_SONATE_IN_GDUR.mid")
print(midi_data.instruments, midi_data.instruments[0].notes[:5])

[Instrument(program=0, is_drum=False, name="up:"), Instrument(program=0, is_drum=False, name="down:")] [Note(start=0.000000, end=0.500000, pitch=71, velocity=86), Note(start=0.500000, end=0.750000, pitch=74, velocity=86), Note(start=0.750000, end=1.000000, pitch=71, velocity=86), Note(start=1.000000, end=1.500000, pitch=67, velocity=86), Note(start=2.500000, end=3.000000, pitch=71, velocity=86)]


c:\Users\zai_2\Desktop\PUC Projeto\venv\lib\site-packages\pretty_midi\pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [3]:
# check for tracks that have more than 4 instruments
midi_files = glob.glob(r"..\midi\downloaded_midi\*")
counter = 1

for midi_file in midi_files:
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    number_of_instruments = len(midi_data.instruments)
    if len(midi_data.instruments) > 4:
        print(f"{counter}. {midi_file} :")
        for instrument_counter, instrument in enumerate(midi_data.instruments):
            print(f"\t{counter}.{instrument_counter+1} " +
                f"{pretty_midi.program_to_instrument_class(instrument.program)}: " +
                f"{pretty_midi.program_to_instrument_name(instrument.program)}")
        counter += 1

1. ..\midi\downloaded_midi\ALL_MY_HEART_THIS_NIGHT_REJOICES.mid :
	1.1 Reed: Oboe
	1.2 Reed: English Horn
	1.3 Strings: Viola
	1.4 Strings: Cello
	1.5 Organ: Church Organ
	1.6 Organ: Church Organ
2. ..\midi\downloaded_midi\DAS_WOHLTEMPERIERTE_CLAVIER_II_FUGA_IX.mid :
	2.1 Pipe: Pan Flute
	2.2 Pipe: Recorder
	2.3 Pipe: Pan Flute
	2.4 Pipe: Recorder
	2.5 Pipe: Pan Flute
	2.6 Pipe: Recorder
	2.7 Pipe: Pan Flute
	2.8 Pipe: Pan Flute
3. ..\midi\downloaded_midi\DAS_WOHLTEMPERIERTE_CLAVIER_I_FUGA_XXIV.mid :
	3.1 Piano: Acoustic Grand Piano
	3.2 Piano: Acoustic Grand Piano
	3.3 Piano: Acoustic Grand Piano
	3.4 Piano: Acoustic Grand Piano
	3.5 Piano: Acoustic Grand Piano
4. ..\midi\downloaded_midi\DER_HIRT_AUF_DEM_FELSEN.mid :
	4.1 Reed: Clarinet
	4.2 Organ: Harmonica
	4.3 Pipe: Pan Flute
	4.4 Piano: Acoustic Grand Piano
	4.5 Piano: Bright Acoustic Piano
5. ..\midi\downloaded_midi\ENFANTILLAGES_PITTORESQUES.mid :
	5.1 Piano: Acoustic Grand Piano
	5.2 Piano: Acoustic Grand Piano
	5.3 Piano: Acou

Pre-treatment of MIDI Data: For all downloaded MIDI files, remove all tracks which Intrument program (intrument sound) falls either on the Piano or Organ categories.

Also, merge all tracks into one, for easier management on the model training phase.

In [4]:
treated_midi = pretty_midi.PrettyMIDI()
treated_piano = pretty_midi.Instrument(
    pretty_midi.instrument_name_to_program('Acoustic Grand Piano'),
    name = 'Piano'
)
for midi_file in midi_files:
    treated_piano.notes = []
    treated_midi.instruments = []
    midi_track = pretty_midi.PrettyMIDI(midi_file)
    valid_instruments = [instrument for instrument in midi_track.instruments \
        if pretty_midi.program_to_instrument_class(instrument.program) in ["Piano", "Organ"]]
    if len(valid_instruments) == 0:
        print(f"{midi_file} has no valid tracks!")
        continue
    for instrument in valid_instruments:
        for note in instrument.notes:
            note.velocity = 60 # all notes will have the same velocity, as it is not really relevant for structure
            treated_piano.notes.append(note)
    if len(treated_piano.notes) < 10:
        print(f"{midi_file} has not enough notes!")
    treated_midi.instruments.append(treated_piano)
    treated_midi.write(midi_file.replace('downloaded', 'treated'))

..\midi\downloaded_midi\DAS_WOHLTEMPERIERTE_CLAVIER_II_FUGA_IX.mid has no valid tracks!
..\midi\downloaded_midi\DIE_KUNST_DER_FUGE_CONTRAPUNCTUS_I.mid has no valid tracks!
..\midi\downloaded_midi\DIE_VGEL.mid has no valid tracks!
..\midi\downloaded_midi\FRHLINGSGLAUBE.mid has no valid tracks!
..\midi\downloaded_midi\MATTHUS_PASSION_CHORAL_O_HAUPT_VOLL_BLUT_UND_WUNDE.mid has no valid tracks!
